In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Install dependencies
!pip install pydub
!apt-get update
!apt-get install -y ffmpeg

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease                        
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease  
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease    
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [68.9 kB]      
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]


In [2]:
import requests
import datetime
import time
import os
import json
import threading
import random
from pydub import AudioSegment

class RadioRecorder:
    def __init__(self, output_dir="recordings"):
        """Initialize the recorder with an output directory."""
        self.output_dir = output_dir
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        # Sample radio stations with stream URLs
        self.radio_stations = {
            "NPR": "https://npr-ice.streamguys1.com/live.mp3",
            "BBC_World_Service": "http://stream.live.vc.bbcmedia.co.uk/bbc_world_service",
            "KEXP": "https://kexp.streamguys1.com/kexp160.aac",
            "WNYC": "https://fm939.wnyc.org/wnycfm",
            "RED_FM" : "https://funasia.streamguys1.com/live9",
            "BIG_CITY" : "https://prclive4.listenon.in/Hindi"
        }
        
        self.metadata_file = os.path.join(output_dir, "recordings_metadata.json")
        self.load_metadata()
        
    def load_metadata(self):
        """Load existing metadata if it exists."""
        if os.path.exists(self.metadata_file):
            with open(self.metadata_file, 'r') as f:
                self.metadata = json.load(f)
        else:
            self.metadata = []
            
    def save_metadata(self):
        """Save metadata to file."""
        with open(self.metadata_file, 'w') as f:
            json.dump(self.metadata, f, indent=4)
            
    def record_stream(self, station_name, duration=30, file_format="mp3"):
        """
        Record a stream for a specified duration.
        
        Args:
            station_name: Name of the radio station to record
            duration: Duration in seconds to record
            file_format: Output file format (mp3 or wav)
        
        Returns:
            Path to the recorded file
        """
        if station_name not in self.radio_stations:
            print(f"Station {station_name} not found in available stations")
            return None
            
        stream_url = self.radio_stations[station_name]
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{station_name}_{timestamp}.{file_format}"
        filepath = os.path.join(self.output_dir, filename)
        
        try:
            # Start streaming and save to file
            print(f"Recording {station_name} for {duration} seconds...")
            
            # Stream chunk by chunk to handle large streams
            response = requests.get(stream_url, stream=True)
            with open(f"{filepath}.temp", 'wb') as f:
                start_time = time.time()
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                    # Check if we've recorded long enough
                    if time.time() - start_time >= duration:
                        break
            
            # Convert temp file to desired format using pydub
            audio = AudioSegment.from_file(f"{filepath}.temp")
            # Trim to exact duration
            audio = audio[:duration * 1000]  # pydub works in milliseconds
            
            if file_format.lower() == "mp3":
                audio.export(filepath, format="mp3")
            else:
                audio.export(filepath, format="wav")
                
            # Clean up temp file
            os.remove(f"{filepath}.temp")
            
            # Record metadata
            recording_metadata = {
                "station": station_name,
                "timestamp": timestamp,
                "duration": duration,
                "format": file_format,
                "filename": filename,
                "stream_url": stream_url
            }
            
            self.metadata.append(recording_metadata)
            self.save_metadata()
            
            print(f"Successfully recorded {station_name} to {filepath}")
            return filepath
            
        except Exception as e:
            print(f"Error recording {station_name}: {e}")
            return None
    
    def record_multiple(self, count=30, duration_range=(30, 90), formats=["mp3"]):
        """
        Record multiple stations randomly.
        
        Args:
            count: Number of recordings to make
            duration_range: Tuple of (min_duration, max_duration) in seconds
            formats: List of formats to use (randomly selected)
        """
        stations = list(self.radio_stations.keys())
        successful_recordings = 0
        
        while successful_recordings < count:
            station = random.choice(stations)
            duration = random.randint(duration_range[0], duration_range[1])
            file_format = random.choice(formats)
            
            print(f"Recording {successful_recordings+1}/{count}: {station} for {duration}s")
            result = self.record_stream(station, duration, file_format)
            
            if result:
                successful_recordings += 1
                # Add a small delay between recordings
                time.sleep(2)
    
    def get_indian_stations(self):
        """Return list of available Indian radio stations."""
        indian_stations = [
            "AIR_FM_Gold", "AIR_Vividh_Bharati", "Radio_Mirchi_98.3", 
            "Radio_City", "Red_FM", "Big_FM", "Fever_104_FM", 
            "Radio_One_India", "Radio_Indigo", "AIR_FM_Rainbow"
        ]
        return [station for station in indian_stations if station in self.radio_stations]
    
    def record_from_country(self, country="india", count=10, duration_range=(30, 90), formats=["mp3"]):
        """
        Record multiple stations from a specific country.
        
        Args:
            country: Country name (currently only "india" is supported)
            count: Number of recordings to make
            duration_range: Tuple of (min_duration, max_duration) in seconds
            formats: List of formats to use (randomly selected)
        """
        if country.lower() == "india":
            stations = self.get_indian_stations()
        else:
            print(f"Country {country} not explicitly supported, using all stations")
            stations = list(self.radio_stations.keys())
            
        successful_recordings = 0
        
        while successful_recordings < count and stations:
            station = random.choice(stations)
            duration = random.randint(duration_range[0], duration_range[1])
            file_format = random.choice(formats)
            
            print(f"Recording {successful_recordings+1}/{count}: {station} for {duration}s")
            result = self.record_stream(station, duration, file_format)
            
            if result:
                successful_recordings += 1
                # Add a small delay between recordings
                time.sleep(2)

# Create a function that can be called directly from the notebook
def record_radio_stations(output_dir="/kaggle/working/recordings", 
                          count=30, 
                          min_duration=30, 
                          max_duration=90, 
                          formats="mp3,wav", 
                          country=None):
    """
    Main function to record radio stations.
    
    Args:
        output_dir: Directory to save recordings
        count: Number of recordings to make
        min_duration: Minimum recording duration in seconds
        max_duration: Maximum recording duration in seconds
        formats: Comma-separated string of formats
        country: Record from specific country (e.g., "india")
    """
    format_list = formats.split(',')
    
    recorder = RadioRecorder(output_dir=output_dir)
    
    if country:
        recorder.record_from_country(
            country=country,
            count=count,
            duration_range=(min_duration, max_duration),
            formats=format_list
        )
    else:
        recorder.record_multiple(
            count=count,
            duration_range=(min_duration, max_duration),
            formats=format_list
        )
    
    print(f"Completed recording {count} audio files.")
    print(f"Metadata saved to {recorder.metadata_file}")
    
    # Create a zip file for easy download from Kaggle
    try:
        import shutil
        zip_path = os.path.join(os.path.dirname(output_dir), "recordings.zip")
        shutil.make_archive(zip_path.replace(".zip", ""), 'zip', output_dir)
        print(f"Created zip archive at {zip_path}")
    except Exception as e:
        print(f"Could not create zip archive: {e}")

In [3]:
# Install dependencies
!pip install requests pydub
!apt-get update && apt-get install -y ffmpeg

# Copy the script code into a cell and run it
# (Copy the entire code from the artifact above)

# Now use the function directly
record_radio_stations(
    output_dir="/kaggle/working/recordings",
    count=30,
    min_duration=30,
    max_duration=90,
    formats="mp3",
    country=None  # Set to None for all stations
)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [68.9 kB]                 
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease                        
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,730 kB]                      
Hit:

## We can use this dataset as a test dataset for an automatic speech recognition (ASR) model

In [3]:
# !pip install openai-whisper ffmpeg
# !apt-get update && apt-get install -y ffmpeg

In [2]:
import whisper
import os
from pathlib import Path

# Load Whisper model (Use 'medium' or 'large' for better accuracy)
model = whisper.load_model("medium", device="cuda")  # GPU enabled

# Path to the uploaded audio dataset in Kaggle
audio_folder = "/kaggle/input/recordings/Final_recordings"
output_folder = "/kaggle/working/transcripts"

# Create output folder
os.makedirs(output_folder, exist_ok=True)

# Process each audio file
for file_path in Path(audio_folder).glob("*"):
    if file_path.suffix in [".mp3", ".wav", ".m4a", ".flac"]:
        print(f"Transcribing: {file_path.name}...")

        # Transcribe audio
        result = model.transcribe(str(file_path))

        # Save transcript
        transcript_path = os.path.join(output_folder, f"{file_path.stem}.txt")
        with open(transcript_path, "w", encoding="utf-8") as f:
            f.write(result["text"])

        print(f"Saved: {transcript_path}")

print("✅ Transcription complete! Transcripts are saved in /kaggle/working/transcripts")


100%|█████████████████████████████████████| 1.42G/1.42G [00:20<00:00, 74.5MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcribing: BIG_CITY_20250309_062022.mp3...
Saved: /kaggle/working/transcripts/BIG_CITY_20250309_062022.txt
Transcribing: NPR_20250309_062413.mp3...
Saved: /kaggle/working/transcripts/NPR_20250309_062413.txt
Transcribing: NPR_20250309_061101.mp3...
Saved: /kaggle/working/transcripts/NPR_20250309_061101.txt
Transcribing: BIG_CITY_20250309_061437.mp3...
Saved: /kaggle/working/transcripts/BIG_CITY_20250309_061437.txt
Transcribing: WNYC_20250309_060226.mp3...
Saved: /kaggle/working/transcripts/WNYC_20250309_060226.txt
Transcribing: NPR_20250309_062838.mp3...
Saved: /kaggle/working/transcripts/NPR_20250309_062838.txt
Transcribing: BIG_CITY_20250309_061725.mp3...
Saved: /kaggle/working/transcripts/BIG_CITY_20250309_061725.txt
Transcribing: WNYC_20250309_060443.mp3...
Saved: /kaggle/working/transcripts/WNYC_20250309_060443.txt
Transcribing: BIG_CITY_20250309_060849.mp3...
Saved: /kaggle/working/transcripts/BIG_CITY_20250309_060849.txt
Transcribing: NPR_20250309_061651.mp3...
Saved: /kaggle/